In [1]:
import pandas as pd
import json

In [2]:
ls

 Volume in drive C is Windows
 Volume Serial Number is 067B-E231

 Directory of C:\Users\haluk\Documents\AI_\Become a Pro\Ottorecommendersystem

10/06/2024  12:39 PM    <DIR>          .
02/02/2024  01:42 PM    <DIR>          ..
10/06/2024  11:50 AM    <DIR>          .ipynb_checkpoints
10/05/2024  02:34 PM        19,580,550 9_OTTO_competition_Recommendation_Systems_HC.ipynb
10/06/2024  12:39 PM            17,038 OTTO_competition_Recommendation_Systems_HC.ipynb
11/06/2023  11:46 AM        95,429,333 sample.csv
11/06/2023  06:45 AM       183,898,350 sample_submission.csv
11/06/2023  06:45 AM       402,090,304 test.jsonl
11/06/2023  06:47 AM    11,307,535,945 train.jsonl
11/09/2023  12:22 PM            16,753 Untitled.ipynb
11/09/2023  09:03 AM        13,744,462 Untitled-Copy1.ipynb
11/09/2023  12:07 PM           451,542 Untitled-Copy2.ipynb
11/11/2023  08:48 PM           453,447 Untitled-Copy3.ipynb
11/10/2023  08:00 AM         3,067,637 Untitled-Copy4.ipynb
              11 File(s) 12,02

In [3]:
# Data is very large 13 GB and there are nearly 13 millions of session data
# I took a sample of 1 million sessions from this data. Each 
# session includes information of visited articles and wether
# article is just clicked/added/ or ordered
data = []
with open('train.jsonl', 'r') as file:
    for line in file:
        record = json.loads(line)
        data.append(record)
dfs = pd.DataFrame(data)        

In [4]:
dfs = dfs.sample(1000000)

## EDA

In [5]:
dfs

,session,events
4103218,4103218,"[{'aid': 834891, 'ts': 1659716929779, 'type': ..."
6323324,6323324,"[{'aid': 1796489, 'ts': 1660118902318, 'type':..."
9594092,9594092,"[{'aid': 1760374, 'ts': 1660736148189, 'type':..."
7857980,7857980,"[{'aid': 1287505, 'ts': 1660402127404, 'type':..."
6131980,6131980,"[{'aid': 1460571, 'ts': 1660069384068, 'type':..."
...,...,...
7339903,7339903,"[{'aid': 1642433, 'ts': 1660310214693, 'type':..."
11406812,11406812,"[{'aid': 965810, 'ts': 1661233916841, 'type': ..."
5440780,5440780,"[{'aid': 817731, 'ts': 1659948728965, 'type': ..."
493319,493319,"[{'aid': 1689170, 'ts': 1659353129508, 'type':..."


In [6]:
dfs.reset_index(inplace = True)
#dfs.drop('Unnamed: 0', axis=1, inplace= True)
dfs.drop('index', axis=1, inplace= True)

## DATA PREPARATION 

In [7]:
# Each session information gives click/add-to-cart/order data starting from a specific article
# We can convert this data into adjacency matrix, where nodes are articles, and edges are click/add-to-cart/order relationship
# "Adjacency" between articles can be quantized by summing up weights of number of clicks/add-to-carts/order placements.
# So, we convert session of visited articles into a graph. 



def get_weight(adj_type) :
    weight = 0;
    if adj_type == 'clicks' :
        weight = 0.1
         #print('clicks: {} '.format(weight))

    elif adj_type == 'carts' :
        weight = 0.3
        #print('carts: {} '.format(weight))
            
    else : #adj_type == 'orders' :
                  
        weight = 0.6
        #print('orders: {} '.format(weight))
    return weight;


def update_neighbors(article_dict, visit_list, articles):
    
    article_id  = article_dict[ 'id' ]
    neighbors   = article_dict[ "neighbor" ]

    
    for i in range(1, len( visit_list) ):
        
        #print(item[i]) 
        vs_aid = int( visit_list[i]['aid'] )
        adj_type = visit_list[i]['type']
        weight = get_weight(adj_type)
        
        
        # we avoid self - loops
        if article_id == vs_aid :
            continue;
        else :
            # check if id is in neighbor list
            if vs_aid in neighbors:
                
                # update/increase the new ID weight 

                #neighbors[vs_aid] += weight
                neighbors[vs_aid] =neighbors[vs_aid]+ weight
                
                
                # we must already have a dictionary with vs_aid in articles_
                neighbor_dict = articles_[vs_aid]
                neighbors_of_vs_id = neighbor_dict["neighbor"]
                #print(f'article id {article_id}')
                #print(neighbor_dict)
                #print(neighbors_of_vs_id)
                
                if article_id in neighbors_of_vs_id :
                    neighbors_of_vs_id[article_id] +=weight
                    #neighbors_of_vs_id[article_id] =neighbors_of_vs_id[article_id]+weight
                    #print(f'{article_id} found and its weight updated')
                    
                    neighbor_dict["neighbor"] = neighbors_of_vs_id;
                    #print(neighbor_dict)
                    #neighbor_dict.update( neighbors_of_vs_id);
                    articles_[vs_aid] = neighbor_dict;
                else :
                    print(f'We could not find {article_id} in {vs_aid} ')
                    #wait = input("Press Enter to continue.")
                    #print('visitor:')
                    #print(articles_[article_id])
                    #print('visited:')
                    #print(articles_[vs_aid])
                    
                    neighbor = {article_id : weight}
                    neighbors_of_vs_id.update(neighbor)
                  
                    #neighbors_of_vs_id[article_id] = weight
                    neighbor_dict['neighbor'] = neighbors_of_vs_id;
                    articles_[vs_aid] = neighbor_dict;
                    print(neighbor_dict)
                    #wait = input("Press Enter to continue.")

                #print(f"The ID {vs_aid} is in the neighbors. Weights UPDATED")
            else:
                #print(f"The ID {vs_aid} is NOT in the neighbors. CREATED")
                # Add the new ID and weight to the dictionary
                
                neighbor = {vs_aid : weight}

                neighbors.update(neighbor)
                
                # create article dictionary with vs_aid, and article_id as neighbor
                #print('new dictionary added to articles')
                
                if vs_aid in articles_ :
                    
                    #print(f'updating {article_id} in {vs_aid} as neighbor ')
                    neighbor_dict = articles_[vs_aid]
                    neighbors_of_vs_id = neighbor_dict["neighbor"]
                    
                    neighbor = {article_id : weight}
                    neighbors_of_vs_id.update(neighbor)
                    
                    #neighbors_of_vs_id[article_id] = weight
                    
                    articles_[vs_aid]["neighbor"] = neighbors_of_vs_id
                
                else :
                    
                    new_dict = {"id": vs_aid, "neighbor": {article_id : weight} }
                    # add new dictionary to articles 
                    articles_[vs_aid] = new_dict
                
 
    article_dict["neighbor"] = neighbors
    
    # ADD new dictionary to articles
    articles_[article_id] = article_dict
    
    return articles_

articles  = []
articles_ = {d["id"]: d for d in articles}
print(articles_)

for visit_list in dfs['events']:
    
    #print(item)
    visit_list = str(visit_list).replace("'", '"')
    visit_list = json.loads(visit_list)
    #print("length: {}".format(len(item)))
    
    #id of the article at the begining of session
    article_id = int( visit_list[0]['aid'] )
    
    #check if this article already exist in article list
    # if exist get neighbors
    #print(article_id)
    # if not exist create empty neighors list
    # Convert the article  list to a dictionary for efficient access
    
    if article_id in articles_:
        #print(f'dictionary {article_id} found! Update the dictionary')
        article_dict = articles_[article_id]
        articles_ = update_neighbors(article_dict, visit_list, articles_)

    else :
        #print(f'dictionary {article_id} NOT found! Create a new dictionary ')
        article_dict = {"id": article_id, "neighbor": {} }
        articles_[article_id] = article_dict
        articles_ = update_neighbors(article_dict, visit_list, articles_)
        
        
        
        # Add the new dictionary to the existing dictionary
        #articles_[article_id] = article_dict
        
        #print(article_dict)


{}


In [8]:
# sort articles based on weights gained by click/cart/orders
for art_id, inner_dict in articles_.items() :
    
    #print(f'article_id {art_id}')
    #print(inner_dict)
    
    sorted_neighbors = {k: v for k, v in sorted(inner_dict['neighbor'].items(), key=lambda item: item[1], reverse=True)}
    inner_dict["neighbor"] = sorted_neighbors
    articles_[art_id]= inner_dict


In [9]:
len(articles_)

1222746

In [10]:
updated_list_of_dicts = list(articles_.values())

## RECOMMENDER MODEL

In [11]:
# Here I updated weights as 1/(1+w). Aggregate weight of nodes 
# at two hops, for example, can be found by summing up
# weigths. I use Bellman-Ford algorithm
# to recommend articles. 


graph = {}
for art_id, inner_dict in articles_.items() :
    # Update weights using a dictionary comprehension
    updated_weights = {neighbor_id: 1 / (1 + weight) for neighbor_id, weight in inner_dict['neighbor'].items()}
    #print(updated_weights)
    graph[art_id] = {'id' : art_id, "neighbor": updated_weights}
    
# max     
# find a recommended list of articles     
    
    

In [12]:
# Implementation of Bellman-Ford algorithm. To get a list of recommended articles. We find "closest articles".
# When we run Bellman-Ford algorithm, we find the shortest path between two articles. Close articles
# recommended first. 

initial_node_id = 1826992
weight = 0;
node_bucket = []
max_recommendation = 20;
nr_of_recommendations = 0;

initial_node = graph[initial_node_id]
node_bucket.append({initial_node_id : weight})

#print(node_list)
# max articles to recommend

neighbors = initial_node["neighbor"]
#print(neighbors)

# if node's neighbors are processed never process it 
# again to avaid loops, if new nodes are being reached
# add them to node_bucket
processed_nodes = set()

#list of nodes that are reached via initial node
reached_nodes  = {} 
#the cost of reaching the first node is 0
reached_nodes[initial_node_id] = 0; 

while len(node_bucket) != 0 :
    
    # pop the first element of list 
    node_to_explore = node_bucket.pop(0)
    node_id = next(iter(node_to_explore.keys()))
    weight = next(iter(node_to_explore.values()))
    
    #print(node)
    node = graph[node_id]
    # add this node to the set of processed nodes
    processed_nodes.add(node_id)
    
    # find nodes that are reachable 
    neighbors = node["neighbor"]
    #print(neighbors)
    
    # add nodes to the reachability list
    # while adding to the list, make sure that
    # - weights are updated 
    # - minimum weight node is added and not duplicated

    for id_, node_weight in neighbors.items() :
        #print(f'Processing... {id_} : {node_weight}')
          
        aggregate_weight = node_weight + weight

        if id_ in reached_nodes : 
        # node is already reachable in different path
        # check if shorter path exists
            ex_path_weight = reached_nodes[id_]
            
            if aggregate_weight < ex_path_weight : 
                #update the pathway to reach the article
                reached_nodes[id_] = aggregate_weight;
            
        else:
            new_reached_node = { id_ : aggregate_weight }
            reached_nodes.update(new_reached_node)
            nr_of_recommendations  = nr_of_recommendations  +1
            
        # if the node is not processed before add it to
        if id_ not in processed_nodes :
            node_bucket.append({id_ : reached_nodes[id_]})
   
        if nr_of_recommendations >= max_recommendation :
            break;

In [13]:
reached_nodes

{1826992: 0,
 1239558: 0.3571428571428571,
 161876: 0.3999999999999999,
 1267119: 0.4166666666666667,
 1100567: 0.4347826086956521,
 1000225: 0.45454545454545453,
 1629111: 0.47619047619047616,
 922576: 0.5263157894736842,
 34687: 0.5263157894736842,
 621352: 0.5555555555555556,
 176969: 0.5882352941176471,
 255379: 0.5882352941176471,
 1052093: 0.625,
 71424: 0.625,
 721034: 0.5793650793650792,
 504876: 0.6666666666666666,
 1636724: 0.6666666666666666,
 79600: 0.6666666666666666,
 1795911: 0.6666666666666666,
 1049910: 0.7142857142857143,
 233195: 0.7142857142857143,
 460141: 0.6944444444444444,
 320528: 0.7796101949025487,
 1842584: 0.8545454545454545,
 1682204: 0.7168458781362008,
 1260075: 0.9007352941176471,
 95488: 0.7843137254901961,
 831165: 0.6620097074642529,
 532388: 0.8553459119496855,
 450505: 0.692242114236999,
 976356: 1.1014492753623188,
 1181236: 1.011494252873563,
 1047083: 1.1904761904761905,
 1127979: 0.8796296296296295,
 1109097: 1.001832417124771,
 528333: 1.14025